## Columbia University
### ECBM E4040 Neural Networks and Deep Learning. Fall 2020.

# ECBM E4040 - Assignment 2 - Task 3: Convolutional Neural Network (CNN)

In this task, you are going to first practice the forward/backward propagation of the convolutional operations with NumPy. After that, we will introduce TensorFlow with which you will create your CNN model for an image classification task. 

## CNNs:
This is one of the good posts describing CNNs:

[https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks/](https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks/)

Convolutional neural networks (CNNs) are highly effective for image processing tasks. 

Remember when one builds a MLP model, each connection is multiplied by its own weight. When the input dimension or the first layer is very large, we need a giant matrix to store the weights. This could easily become a problem in image processing since the dimension of a vectorized image could easily exceed 1000 (consider CIFAR-10 which has images of shape 32×32=1024, yet the resolution is so low). 

In CNN, the weights are shared: the same filter (also known as 'weights' or 'kernel') moves over the input, and at each position an output value is calculated. This means that the same weights are repetitively applied to the entire input, therefore saving a lot of memory.

![Illustration of the CNN](./utils/notebook_images/task3_1.jpg)
Image source: [here](https://developer.apple.com/library/content/documentation/Performance/Conceptual/vImage/ConvolutionOperations/ConvolutionOperations.html)

__Convolution:__  In the picture above, the input is a 7-by-7 image, and the filter is shown as a blue 3-by-3 grid. The filter overlaps with the top-left corner of the input, and we perform an element-wise multiplication followed by a summation, then put the sum into the output matrix. The filter then moves several pixels right, covering a new input area so a new sum can be derived.

__Training:__ One thing to remember is that there would be a lot of filters for each layer in a CNN, and the goal of training is to find the best filters for your task. Each filter tries to capture one specific feature. Typically, in the first convolutional layer which directly looks at your input, the filters try to capture information about color and edges which we know as local features; in higher layers, due to the effect of max-pooling, the receptive-fields of filters becomes large so more global and complex features can be detected. 

__Architecture:__ For classification tasks, a CNN usually starts with convolution followed by max-pooling. After that, the feature maps will be flattened so that we could append fully connected layers. Common activation functions include ReLu, ELU in the convolution layers, and softmax in the fully connected layers (to calculate the classification scores).

---

### Terminology

* __Convolution__: element-wise multiplication followed by summation of your input and one of your filters in the CNN context.
* __Filter/kernel/weights__: a grid or a set of grids typically smaller than your input size that moves over the input space to generate output. Each filter captures one type of feature.
* __Feature/feature maps__: the output of a hidden layer. Think of it as another representation of your data. 
* __Pooling__: an downsampling operation that joins local information together, so the higher layers' receptive fields can be bigger. The most seen pooling operation is max-pooling, which outputs the maximum of all values inside the pool.
* __Flatten__: a junction between convolution layers and fully connected layers. Used to turn 2-D feature maps into 1-D. For tasks such as image segmentation where the output also needs to be 2-D, this won't be used.
* __Border mode__: usually refers to 'VALID' or 'SAME'. Under 'VALID' mode, only when the filter and the input fully overlap can a convolution be conducted; under 'SAME' mode, the output size is the same as the input size (only when the stride is 1), and when the filter and the input don't fully overlap (happens at the edge/corner of input) we pad zeroes (or other designated numbers) and then do convolution.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
print(tf.__version__)

2.3.0-dev20200530


In [3]:
tf.test.gpu_device_name()

''

## Part 1: Getting a sense of convolution

### conv2d feedforward

Implement a NumPy naive 2-D convolution feedforward function. We ask you to simply do the element-wise multiplication and summation. Do not worry about the efficiency of your functions. Use as many loops as you like.

<span style="color:red">__TODO:__</span> Complete the function __conv2d_forward__ in __utils/layer_funcs.py__. After that, run the following cell blocks in Jupyter notebook, which will give the output of your convolution function. Detailed instructions have been given in the comments of __layer_func.py__. __The instructors will look at the output to give credits for this task__.

In [4]:
# tf 2.2.0 implementation
from utils.layer_funcs import conv2d_forward
# Set test parameters.
x_shape = (2, 5, 5, 3) #(batch, height, width, channels)
w_shape = (3, 3, 3, 5) #(filter_height, filter_width, channels, num_of_filters)
channels = w_shape[-1]

x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num=channels)
pad = 1
stride = 2
your_feedforward = conv2d_forward(x, w, b, pad, stride)

print("Your feedforward result (size :{}) is: ".format(your_feedforward.shape))
print(your_feedforward)

Your feedforward result (size :(2, 3, 3, 5)) is: 
[[[[-0.17767004 -0.10534459 -0.03301913  0.03930632  0.11163177]
   [-0.14008665 -0.0678814   0.00432385  0.0765291   0.14873435]
   [-0.09471101 -0.0207628   0.05318542  0.12713363  0.20108184]]

  [[ 0.06484073  0.14191425  0.21898778  0.2960613   0.37313483]
   [ 0.1523525   0.23228839  0.31222428  0.39216017  0.47209606]
   [ 0.04323099  0.12273866  0.20224632  0.28175398  0.36126164]]

  [[-0.08652509 -0.00608585  0.0743534   0.15479265  0.23523189]
   [-0.14941751 -0.06504157  0.01933437  0.10371031  0.18808625]
   [-0.19829711 -0.1162351  -0.03417309  0.04788891  0.12995092]]]


 [[[ 0.38218171  0.46803015  0.55387859  0.63972704  0.72557548]
   [ 0.54755735  0.64004708  0.73253681  0.82502655  0.91751628]
   [ 0.2622959   0.3497671   0.43723831  0.52470951  0.61218071]]

  [[ 0.44821747  0.54557548  0.64293349  0.7402915   0.8376495 ]
   [ 0.49921717  0.60957979  0.7199424   0.83030502  0.94066763]
   [ 0.12234048  0.22213263  0

In [5]:
######################################################
# Verification/checking code. Do not modify it       #
######################################################

X_tf = tf.constant(x, shape=x_shape)
w_tf = tf.constant(w, shape=w_shape)
b_tf = tf.constant(b, shape=channels)

def conv2d_forward_2(x, w, b, stride):
    # stride in tf.nn.conv2d is in the format: [1, x_movement, y_movement, 1]
    feedforward = tf.nn.conv2d(x, w, [1, stride, stride, 1], padding = "SAME")
    # add bias to the conv network
    feedforward = tf.nn.bias_add(feedforward, b)
    return feedforward
print("Is your feedforward correct? {}".format(np.allclose(your_feedforward, conv2d_forward_2(X_tf, w_tf, b_tf, stride))))

Is your feedforward correct? True


### conv2d backpropagation (optional, bonus +10 points)

<p style="color:red">This function is optional, but a bonus 10 points will be given if you solve it correctly.</p>

Implement a NumPy naive 2-D convolution backpropagation function. Again, don't worry about the efficiency.

<span style="color:red">__TODO:__</span> Complete the function __conv2d_backward__ in __utils/layer_funcs.py__. After that, run the following cell blocks, which will give the output of your backpropagation. Detailed instructions have been given in the comments of __layer_func.py__. __We need to judge your output to give you credits__.

In [ ]:
from utils.layer_funcs import conv2d_backward
# Set test parameters. Please don't change it.
np.random.seed(234)
d_top = np.random.normal(size=your_feedforward.shape)
your_dw, your_db, d_w_shape = conv2d_backward(d_top, x, w, b, pad, stride)

print("Your weights' gradients result (size :{}) is: ".format(d_w_shape))
print(your_dw)
print("Your biases' gradients result is: ")
print(your_db)

In [ ]:
####################################################
# Verification/checking code. Don't change it.     #
####################################################
d_top_tf = tf.constant(d_top, shape=your_feedforward.shape)
def conv2d_backward_2(x, w, b, d, stride):
    # stride in tf implementation is in the format: [1, x_movement, y_movement, 1]

    dw_tf =  tf.compat.v1.nn.conv2d_backprop_filter(x, w, d, [1, stride, stride, 1], padding = "SAME")
    with tf.GradientTape() as g:
        g.watch(b)
        y = conv2d_forward_2(X_tf, w_tf, b, stride) * d
    dy_dx = g.gradient(y, b)
    return dw_tf, dy_dx

print("Is your weights' gradients correct? {}".format(np.allclose(your_dw, conv2d_backward_2(X_tf, w_shape, b_tf, d_top_tf, stride)[0])))
print("Is your biases' gradients correct? {}".format(np.allclose(your_db, conv2d_backward_2(X_tf, w_shape, b_tf, d_top_tf, stride)[1])))

### max pool feedforward

Implement a NumPy naive max pool feedforward function. We ask you to simply find the max in your pooling window. Also, don't need to worry about the efficiency of your function. Use loops as many as you like.

<span style="color:red">__TODO:__</span> Finish the function __max_pool_forward__ in __utils/layer_funcs.py__. After that, run the following cell blocks, which will give the output of your max pool function. Detailed instructions have been given in the comments of __layer_func.py__. __We need to judge your output to give you credits__.

In [6]:
from utils.layer_funcs import max_pool_forward

# Set test parameters.
x_shape = (2, 5, 5, 3) #(batch, height, width, channels)
x = np.linspace(-0.5, 0.5, num=np.prod(x_shape)).reshape(x_shape)
pool_size = 2
stride = 2

your_feedforward = max_pool_forward(x, pool_size, stride)

print("Your feedforward result (size :{}) is: ".format(your_feedforward.shape))
print(your_feedforward)

Your feedforward result (size :(2, 2, 2, 3)) is: 
[[[[-0.37919463 -0.37248322 -0.36577181]
   [-0.33892617 -0.33221477 -0.32550336]]

  [[-0.17785235 -0.17114094 -0.16442953]
   [-0.13758389 -0.13087248 -0.12416107]]]


 [[[ 0.12416107  0.13087248  0.13758389]
   [ 0.16442953  0.17114094  0.17785235]]

  [[ 0.32550336  0.33221477  0.33892617]
   [ 0.36577181  0.37248322  0.37919463]]]]


In [7]:
####################################################
# Verification/checking code. Don't change it.     #
####################################################
X_tf = tf.constant(x, shape=x_shape)

def maxpool_forward_2(x, pool_size, stride):
    maxpool_forward = tf.nn.max_pool(x, [1, pool_size, pool_size, 1], [1, stride, stride, 1], padding='VALID')
    return maxpool_forward

## Print validation result
print("Is your feedforward correct? {}".format(np.allclose(your_feedforward, maxpool_forward_2(X_tf, pool_size, stride))))

Is your feedforward correct? True


### max pool backpropogation (optional, bonus +10 points)

<p style="color:red">This function is optional, but a bonus 10 points will be given if you solve it correctly.</p>

Implement a Numpy naive max pooling backpropagation function. Again, don't worry about the efficiency.

<span style="color:red">__TODO:__</span> Finish the function __max_pool_backward__ in __utils/layer_funcs.py__. After that, run the following cell blocks, which will give the output of your backpropagation. Detailed instructions have been given in the comments of __layer_func.py__. __We need to judge your output to give you credits__.

In [ ]:
from utils.layer_funcs import max_pool_backward

# Set test parameters. Please don't change it.
np.random.seed(234)
dout = np.random.normal(size=your_feedforward.shape)
dx = max_pool_backward(dout, x, pool_size, stride)

print("Your inputs' gradients result (size :{}) is: ".format(dx.shape))
print(dx)

In [ ]:
#######################################
# Checking code. Don't change it.     #
#######################################
d_out_tf = tf.constant(dout, shape=your_feedforward.shape)

def max_pool_backward_2(x, d):
    with tf.GradientTape() as g:
        g.watch(x)
        y = maxpool_forward_2(X_tf, pool_size, stride) * d
    dy_dx = g.gradient(y, x)
    return dy_dx
# ## Print validation result
print('*'*50)
print("Is your inputs' gradients correct? {}".format(np.allclose(dx, max_pool_backward_2(X_tf, d_out_tf))))

## Part 2: TensorFlow CNN

In this part we will construct the CNN in TensorFlow. We will implement a CNN similar to the LeNet structure.

TensorFlow offers many useful resources and functions which help developers build the net in a high-level fashion, such as functions in the `layer` module. However, we will build the network **by ourself** for this homework for better understanding. By utilizing functions in `tf.nn` that exist for Neural Network structuring and training, we can build out our own layers and network modules rather quickly.

Also, we will introduce a visualization tool called Tensorboard. You can use TensorBoard to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data that pass through it.

Resources and References: <br>
* [TensorBoard: Visualizing Learning](https://www.tensorflow.org/get_started/summaries_and_tensorboard)<br>
* [Convolutional Neural Networks (LeNet) - DeepLearning 0.1 documentation](http://deeplearning.net/tutorial/lenet.html)<br>
* [LeNet-5, convolutional neural networks](http://yann.lecun.com/exdb/lenet/)

### Quick guide for Tensorboard

Tensorboard is a powerful tool provided by TensorFlow. It allows developers to check their graph and trend of parameters. This guide will give you a basic under standing on how to set up Tensorboard graph in your code, start tensorboard on your local machine/GCP instance and how to access the interface.

For complete instructions, check the official guide on Tensorflow web site [here](https://www.tensorflow.org/get_started/summaries_and_tensorboard).

### How to start tensorboard

#### Local

To start your Tensorboard on your local machine, you need to specify a log directory for the service to fetch the graph. For example, in your command line, type:

```shell
$ tensorboard --logdir="~/log"
```

Then, Tensorboard will start running. By default, it will be running on port 6006:

``` shell
TensorBoard 1.8.0 at http://localhost:6006 (Press CTRL+C to quit)
```

Make sure Tensorboard is running, you can visit http://localhost:6006 In your browser and you should be able to see the main page of Tensorboard. If the page is shown as below, it means Tensorboard is running correctly. The report is due to lack of event file, but we can just leave it there for now.

![Tensorboard_1](./utils/notebook_images/task3_2_1.png)

#### GCP

To set up the Tensorboard on GCP is the same as above. However, we're not able to check the Tensorboard UI directly through our browser. In order to visit the page through our local browser, we should link the port of our local machine to the port on GCP. It is similar to what we did previously for Jupyter Notebook.

In the command line on your local machine, type:

```shell
$ gcloud compute ssh --ssh-flag="-L 9999:localhost:9999 -L 9998:localhost:6006" "ecbm4040@YOUR_INSTANCE"
```

 This will bind your port of your local machine to the port on GCP instance. In this case, your local port 9999 is binded with 9999 on GCP, while local port 9998 is binded with 6006 on GCP. You can change whatever port you like as long as it does not confilct with your local services.

After connecting to GCP using the command, you will be able to see the result page.



### Export Tensorboard events into log directory

To write summary data for visualization in TensorBoard, we should use class `tf.summary`. This class will save your network graph sturcuture and all the variable summary. 

For example, the following code from the LeNet_trainer.py will save all the parameter summary and marked with iteration_total. These data will be displayed in the Tensorboard later on.

```python
# ... previous code ...
# ...
				# Set up summary writers to write the summaries to disk in a different logs directory:
                def summary(self):
                    self.current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
                    self.train_log_dir = 'logs/gradient_tape/' + self.current_time + '/train'
                    self.test_log_dir = 'logs/gradient_tape/' + self.current_time + '/test'
                    self.train_summary_writer = tf.summary.create_file_writer(self.train_log_dir)
                    self.test_summary_writer = tf.summary.create_file_writer(self.test_log_dir)
                
                # Use tf.summary.scalar() to log metrics (loss and accuracy) during training/testing within the scope of
                # the summary writers to write the summaries to disk.
                def train_epoch(self, epoch):
                    ...previous code...
                    for images, labels in train_ds:
                    self.train_step(images, labels)
                    with self.train_summary_writer.as_default():
                        tf.summary.scalar('loss', self.train_loss.result(), step=epoch)
                        tf.summary.scalar('accuracy', self.train_accuracy.result(), step=epoch)

                    for test_images, test_labels in test_ds:
                        self.test_step(test_images, test_labels)

                    with self.test_summary_writer.as_default():
                        tf.summary.scalar('loss', self.test_loss.result(), step=epoch)
                        tf.summary.scalar('accuracy', self.test_accuracy.result(), step=epoch)
```



### Check the graph and summary in Tensorboard

After executing the program once, you should able to see the metrics displayed in the tensorboard. 

![Tensorboard_2](./utils/notebook_images/Task3_2_2_metrics.png)


Also, you may able zoom in or zoom out or click into the layer block to check all the variables and tensor operations in the graph, check the trend of the variables and the distribution of those in Scalar, Distributions and Histograms. You may explore the tensorboard by yourself and take advantage to it for debuging the nerwork structure.

<span style="color:red">__TODO:__</span> You will try to achieve your own CNN model that has similar structure to LeNet, show the model graph in tensorboard, and get a model with __90% or higher accuracy__ using the data we provide you.

An example code is included in __utils/neuralnets/cnn/model_LeNet.py__. This sample is used as a guide line for how to build a Neural Net model in Tensorflow. Feel free to copy or utilize the code we give you.

<span style="color:red">__TODO:__</span>
1. Edit the file __utils/neuralnets/cnn/my_model_LeNet.py__. Create your own CNN that is based on the LeNet structure to achieve at least **90%** test accuracy.
2. Print out the training process and the best validation accuracy, save the `.meta` model in __model/__ folder.
3. Attach a screenshot of your tensorboard graph in the markdown cell below. Double click the cell and replace the example image with your own image. Here is a [Markdown Cheetsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#images) that may also help.

__Hint__: 
1. You can copy and edit the code from `model_LeNet.py`


# Sequential implementation.

In [8]:
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
import datetime

In [9]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [10]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_train[0].shape, 'image shape')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(28, 28, 1) image shape


<span style="color:red">__TODO:__</span> Show accuracy and attach the tensorboard graph.

In [11]:
from utils.neuralnets.cnn.LeNet_trainer import MyLeNet_trainer

trainer = MyLeNet_trainer(x_train,y_train,x_test,y_test)
trainer.run()

Training Epoch 1
Loss: 1.7867103815078735, Accuracy: 67.4816665649414, Test Loss: 1.690656304359436, Test Accuracy: 77.06999969482422
Training Epoch 2
Loss: 1.668280005455017, Accuracy: 79.2550048828125, Test Loss: 1.6564232110977173, Test Accuracy: 80.41999816894531
Training Epoch 3
Loss: 1.647902011871338, Accuracy: 81.31500244140625, Test Loss: 1.6554571390151978, Test Accuracy: 80.54999542236328
Training Epoch 4
Loss: 1.6375353336334229, Accuracy: 82.30999755859375, Test Loss: 1.6451575756072998, Test Accuracy: 81.45999908447266
Training Epoch 5
Loss: 1.603250503540039, Accuracy: 85.78666687011719, Test Loss: 1.5789718627929688, Test Accuracy: 88.16999816894531
Training Epoch 6
Loss: 1.5670949220657349, Accuracy: 89.36833190917969, Test Loss: 1.5740430355072021, Test Accuracy: 88.76000213623047
Training Epoch 7
Loss: 1.560694694519043, Accuracy: 90.03166198730469, Test Loss: 1.5695619583129883, Test Accuracy: 89.0999984741211
Training Epoch 8
Loss: 1.5565170049667358, Accuracy: 90.

**Notes: Don't forget to set up a firewall for Tensorboard just like the way we did for jupyter notebook. This time the port is 6006.**

In [12]:
%load_ext tensorboard
# Load tensorboard on google cloud with specific port
%tensorboard --logdir=logs/gradient_tape/ --port=6006 --bind_all

In [ ]:
# Save the entire model 
!mkdir -p saved_model
trainer.save('model/my_model') 

![Tensorboard_my_LeNet](./utils/notebook_images/Tensorboard_my_LeNet.png)

In [7]:
trainer.model.save('model/my_model')

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model/my_model/assets
